# Task 2: Medical dataset

In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression, Ridge, HuberRegressor, SGDClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.kernel_approximation import RBFSampler, Nystroem
from sklearn.pipeline import Pipeline
import sklearn.metrics as metrics

import utils
import time

pd.options.display.max_columns = 40

---
## 0. Preliminary Analysis

In [81]:
# df_extract = utils.extract_patient_feature(feat_train, [lambda x: x.min(axis=1), lambda x: x.max(axis=1)])
# df_extract
# feat_train = pd.read_csv("./train_features_filled.zip", header=None, index_col=0, names=utils.flattened_columns())
# feat_train
pd.concat([feat_train, df_extract], axis=1).sort_index(axis=1)

ABPd                                                                    \
          0     1     2     3     4     5     6     7     8     9    10    11   
1      61.0  62.5  59.0  49.5  48.0  51.0  46.0  47.0  47.0  44.0  49.0  39.0   
10     90.0  90.0  92.0  99.0  76.0  82.0  73.0  64.0  85.0  97.0  80.0  78.0   
100    74.0  55.0  54.0  61.0  65.0  63.0  68.0  63.0  61.0  59.0  64.0  67.0   
1000   66.0  66.0  63.0  66.0  55.5  62.5  69.0  69.0  65.0  63.0  57.0  55.0   
10000  49.5  49.5  43.0  56.0  47.5  51.0  51.0  38.0  47.0  51.0  52.5  44.0   
...     ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
9993   80.5  80.5  80.5  74.0  65.0  65.0  67.5  66.5  69.0  73.0  70.0  68.0   
9995    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
9996   65.5  61.0  58.0  61.0  62.0  57.0  49.5  46.5  52.0  55.0  53.5  50.5   
9998   50.0  50.0  56.0  52.0  66.0  56.5  49.0  49.0  58.0  57.0  49.0  59.0   
9999   85.0  85.0  89.0  89.0  89.0  89.0  89.0  89.0  89.0  89.0  89.0  89.0   

                      ABPm                                     ...        WBC  \
      feat00 feat01      0      1      2      3      4      5  ...          8   
1       39.0   62.5   84.0   81.0   74.0   66.0   63.0   68.0  ...   4.700000   
10      64.0   99.0  100.0  100.0  108.0  120.0  101.0   97.0  ...  11.500000   
100     54.0   74.0  103.0   73.0   71.0   82.0   85.0   81.0  ...  12.500000   
1000    55.0   69.0   82.0   82.0   80.0   82.0   71.0   84.0  ...  19.600000   
10000   38.0   56.0   68.0   68.0   59.0   81.0   68.0   73.0  ...   7.888889   
...      ...    ...    ...    ...    ...    ...    ...    ...  ...        ...   
9993    65.0   80.5   93.0   93.0   92.0   84.0   73.0   72.0  ...   9.700000   
9995     NaN    NaN   94.0   94.0   98.0  110.0  100.0  101.0  ...  13.600000   
9996    46.5   65.5   84.0   78.0   73.0   80.0   82.0   76.0  ...        NaN   
9998    49.0   66.0   72.0   72.0   76.0   80.0  100.0   92.0  ...  20.000000   
9999    85.0   89.0  111.0  111.0  116.0   98.0  104.0   78.0  ...   5.100000   

                                              pH                          \
               9    10    11 feat00 feat01     0      1      2         3   
1       4.700000   4.7   4.7    4.7    6.3  7.33  7.330  7.370  7.370000   
10     11.500000  11.5  11.5   11.5   11.5   NaN    NaN    NaN       NaN   
100    12.500000  12.5  12.5   12.5   12.5   NaN    NaN    NaN       NaN   
1000   19.600000  19.6  19.6   19.6   19.6  7.30  7.300  7.300  7.300000   
10000   7.944444   8.0   8.0    7.5    8.0  7.35  7.350  7.385  7.420000   
...          ...   ...   ...    ...    ...   ...    ...    ...       ...   
9993    9.700000   9.7   9.7    8.4    9.7   NaN    NaN    NaN       NaN   
9995   13.600000  13.6  13.6   13.6   13.6   NaN    NaN    NaN       NaN   
9996         NaN   NaN   NaN    NaN    NaN  7.41  7.385  7.360  7.376667   
9998   20.000000  20.0  20.0   20.0   20.0   NaN    NaN    NaN       NaN   
9999    5.100000   5.1   5.1    5.1    5.1   NaN    NaN    NaN       NaN   

                                                                            \
              4      5         6         7      8     9    10    11 feat00   
1      7.410000  7.406  7.402000  7.398000  7.394  7.39  7.39  7.39   7.33   
10          NaN    NaN       NaN       NaN    NaN   NaN   NaN   NaN    NaN   
100         NaN    NaN       NaN       NaN    NaN   NaN   NaN   NaN    NaN   
1000   7.300000  7.300  7.300000  7.300000  7.300  7.30  7.30  7.30   7.30   
10000  7.390000  7.400  7.400000  7.400000  7.400  7.40  7.40  7.40   7.35   
...         ...    ...       ...       ...    ...   ...   ...   ...    ...   
9993        NaN    NaN       NaN       NaN    NaN   NaN   NaN   NaN    NaN   
9995        NaN    NaN       NaN       NaN    NaN   NaN   NaN   NaN    NaN   
9996   7.393333  7.410  7.416667  7.423333  7.430  7.43  7.43  7.43   7.36   
9998        NaN    NaN       NaN       NaN    NaN   

In [4]:
feat_train = pd.read_csv("./train_features.csv")
labl_train = pd.read_csv("./train_labels.csv")
feat_test = pd.read_csv("./test_features.csv")

In [82]:
utils.get_score("./train_labels.csv", "./recall_train_ds-tree-ensemble-CV.zip")

[0.9690232025998732, 0.9464191937644624, 0.8643301821714431, 0.8896631134062567, 0.8638892559037351, 0.8787747094011449, 0.9724417750621056, 0.9083477682667451, 0.975375408993231, 0.9868908304653337]
[0.7054040284153551, 0.7765732325442096, 0.5958148159467647, 0.8359770663905393]
0.9255155440034333 0.9270635979932265 0.7284422858242172


0.8603404759402924

In [5]:
feat_train_reformed = utils.patient_feat_flatten(feat_train)
feat_test_reformed = utils.patient_feat_flatten(feat_test)

In [ ]:
feat_train_filled = utils.fill_interp(feat_train_reformed)
# feat_train_filled = pd.read_csv("./intermediate.zip", header=None, names=feat_train_reformed.columns)

In [237]:
feat_train_filled.to_csv("./intermediate.zip", header=False, compression="zip")

In [238]:
feat_train_fill0 = feat_train_filled.fillna(0.0, inplace=False)

---
## 1. Training

In [6]:
feat_train_filled = pd.read_csv("./train_features_filled.zip", header=None, names=feat_train_reformed.columns)
feat_train_filled = feat_train_filled.groupby(axis=1, level=0).apply(lambda df: df / np.nanmedian(df.to_numpy().flatten()))
feat_train_fill0 = feat_train_filled.fillna(0.0, inplace=False)

In [9]:
feat_train_filled

EtCO2                                                              \
             0         1         2         3         4         5         6    
1           NaN       NaN       NaN       NaN       NaN       NaN       NaN   
10          NaN       NaN       NaN       NaN       NaN       NaN       NaN   
100         NaN       NaN       NaN       NaN       NaN       NaN       NaN   
1000   0.787879  0.787879  0.893939  0.909091  0.984848  1.075758  1.121212   
10000       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
...         ...       ...       ...       ...       ...       ...       ...   
9993        NaN       NaN       NaN       NaN       NaN       NaN       NaN   
9995        NaN       NaN       NaN       NaN       NaN       NaN       NaN   
9996        NaN       NaN       NaN       NaN       NaN       NaN       NaN   
9998        NaN       NaN       NaN       NaN       NaN       NaN       NaN   
9999        NaN       NaN       NaN       NaN       NaN       NaN       NaN   

                                                              PTT            \
             7         8         9         10        11        0         1    
1           NaN       NaN       NaN       NaN       NaN       NaN       NaN   
10          NaN       NaN       NaN       NaN       NaN  0.879747  0.879747   
100         NaN       NaN       NaN       NaN       NaN  0.661392  0.661392   
1000   1.121212  1.075758  0.909091  0.863636  0.878788       NaN       NaN   
10000       NaN       NaN       NaN       NaN       NaN  0.813291  0.813291   
...         ...       ...       ...       ...       ...       ...       ...   
9993        NaN       NaN       NaN       NaN       NaN       NaN       NaN   
9995        NaN       NaN       NaN       NaN       NaN  1.756329  1.756329   
9996        NaN       NaN       NaN       NaN       NaN       NaN       NaN   
9998        NaN       NaN       NaN       NaN       NaN  1.098101  1.098101   
9999        NaN       NaN       NaN       NaN       NaN  1.151899  1.151899   

                                                                   ...  \
             2         3         4         5         6         7   ...   
1           NaN       NaN       NaN       NaN       NaN       NaN  ...   
10     0.879747  0.879747  0.879747  0.879747  0.879747  0.879747  ...   
100    0.661392  0.661392  0.661392  0.661392  0.661392  0.661392  ...   
1000        NaN       NaN       NaN       NaN       NaN       NaN  ...   
10000  0.833333  0.853376  0.873418  0.893460  0.913502  0.933544  ...   
...         ...       ...       ...       ...       ...       ...  ...   
9993        NaN       NaN       NaN       NaN       NaN       NaN  ...   
9995   1.756329  1.756329  1.756329  1.756329  1.756329  1.756329  ...   
9996        NaN       NaN       NaN       NaN       NaN       NaN  ...   
9998   1.098101  1.098101  1.098101  1.098101  1.098101  1.098101  ...   
9999   1.151899  1.151899  1.151899  1.151899  1.151899  1.151899  ...   

           ABPs                                                              \
             5         6         7         8         9         10        11   
1      0.883333  0.933333  1.008333  0.983333  0.916667  1.033333  0.850000   
10     1.025000  0.941667  0.950000  1.133333  1.166667  1.091667  1.133333   
100    0.941667  1.083333  0.941667  0.941667  0.916667  0.808333  0.966667   
1000   1.200000  1.458333  1.541667  1.266667  1.266667  1.116667  1.041667   
10000  1.083333  1.100000  0.816667  0.925000  1.266667  1.175000  1.100000   
...         ...       ...       ...       ...       ...       ...       ...   
9993   0.750000  0.766667  0.766667  0.791667  0.800000  0.841667  0.841667   
9995   1.525000  1.533333  1.508333  1.558333  1.558333  1.641667  1.450000   
9996   0.883333  0.733333  0.666667  0.800000  0.833333  0.833333  0.791667   
9998   1.216667  0.991667  1.025000  1.183333  1.191667  1.050000  1.300000   
9999   0.908333  0.750000  0.916667  1.0250

In [341]:
feat_train_mat = np.concatenate((feat_train_fill0.to_numpy(), np.ones((feat_train_reformed.shape[0], 1))), axis=1)

In [424]:
rbf_kernel = RBFSampler(gamma=2, n_components=10000, random_state=1)
X_rbf_feat = rbf_kernel.fit_transform(feat_train_mat)

In [425]:
X_rbf_feat

array([[ 0.00124752, -0.00560517, -0.01202498, ..., -0.00682438,
         0.00972497, -0.00853319],
       [-0.01294071,  0.01386927,  0.01289027, ...,  0.0123179 ,
         0.0019333 , -0.01055929],
       [-0.0102816 ,  0.00391926, -0.00992195, ..., -0.01391045,
         0.00951419, -0.01131782],
       ...,
       [-0.00202312,  0.00287168,  0.00664406, ...,  0.01040662,
         0.00135352, -0.00219292],
       [-0.01303557, -0.00146097, -0.00506871, ...,  0.01258362,
        -0.00579549, -0.01278685],
       [-0.00729411, -0.00982529, -0.01187047, ...,  0.01411375,
        -0.01183319, -0.00734779]])

In [434]:
rbf_kernel.transform(feat_test_mat)

array([[ 0.01158714, -0.01357454,  0.00763057, ..., -0.00773793,
         0.00022349, -0.00511733],
       [ 0.01413437,  0.01396206,  0.00843097, ...,  0.01198258,
         0.01410292, -0.00505409],
       [-0.01108782, -0.00990463,  0.01413618, ...,  0.01410968,
         0.01140916,  0.01239361],
       ...,
       [ 0.00422473,  0.0122155 ,  0.01070024, ..., -0.00549259,
         0.00658475,  0.01164897],
       [-0.00915065, -0.00669379,  0.01282676, ...,  0.00559994,
         0.0096538 ,  0.01343743],
       [ 0.00738116, -0.00975196,  0.01136088, ..., -0.00343837,
         0.01059531,  0.01412997]])

### Logistic regression

In [383]:
tstart = time.time()

# logit_lin = LogisticRegression(C=1, fit_intercept=False, solver="sag", max_iter=1000).fit(feat_train_mat, labl_train.iloc[:, 1])
logit_lin = SGDClassifier(loss="log", fit_intercept=False, max_iter=100, alpha=1, tol=1e-7).fit(feat_train_mat, labl_train.iloc[:, 11])

tend_lin = time.time()
print("Linear classifier training time: {}".format(tend_lin - tstart))

# logit_nonlin = LogisticRegression(C=1, fit_intercept=False, solver="sag", max_iter=500).fit(X_rbf_feat, labl_train.iloc[:, 11])
logit_nonlin = SGDClassifier(loss="log", fit_intercept=False, max_iter=500, tol=1e-7).fit(X_rbf_feat, labl_train.iloc[:, 11])

tend_nonlin = time.time()
print("Nonlinear kernels training time: {}".format(tend_nonlin - tstart))

Linear classifier training time: 0.9513750076293945
Nonlinear kernels training time: 193.39767217636108


In [384]:
prob_lin = logit_lin.predict_proba(feat_train_mat)[:, 1]
prob_nonlin = logit_nonlin.predict_proba(X_rbf_feat)[:, 1]
prob_lin, prob_nonlin

(array([0.09374422, 0.05322627, 0.05382918, ..., 0.07817172, 0.05483041,
        0.05807976]),
 array([0.45822555, 0.56198689, 0.42468296, ..., 0.47526155, 0.45917428,
        0.4713796 ]))

In [386]:
metrics.roc_auc_score(labl_train.iloc[:, 11], prob_lin), metrics.roc_auc_score(labl_train.iloc[:, 11], prob_nonlin)

(0.5178642040247159, 0.8580173933788627)

### Support-Vector-Machine classification

In [363]:
svm_lin = LinearSVC(fit_intercept=False).fit(feat_train_mat, labl_train.iloc[:, 11])
svm_nonlin = LinearSVC(fit_intercept=False).fit(X_rbf_feat, labl_train.iloc[:, 11])

E:\Miniconda3\envs\intro-ml\lib\site-packages\sklearn\svm\_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


In [364]:
func_lin = svm_lin.decision_function(feat_train_mat)
prob_svm_lin = 1/(1 + np.exp(-func_lin))

In [367]:
metrics.roc_auc_score(labl_train.iloc[:, 11], prob_svm_lin)

0.6381671930792757

In [52]:
vital_predictor = Lasso(alpha=1, fit_intercept=True).fit(feat_na_filled, labl_train["LABEL_RRate"])

In [53]:
0.5 + 0.5*np.maximum(0, metrics.r2_score(labl_train["LABEL_RRate"], vital_predictor.predict(feat_na_filled)))

0.636415255250034

In [ ]:
logistic_rbf = Pipeline([("rbf_approx", RBFSampler(gamma=2, n_components=1, random_state=1)), 
                         ("sgd_log", SGDClassifier(loss="log", fit_intercept=False, max_iter=500))])
logistic_rbf.fit(

---

## Checking Results

In [451]:
X_rbf_feat[1,:] @ X_rbf_feat[1,:]

0.9918226790121505

In [426]:
feat_test_mat = np.concatenate((feat_test_fill0.to_numpy(), np.ones((feat_test_reformed.shape[0], 1))), axis=1)
X_rbf_feat_test = rbf_kernel.transform(feat_test_mat)

prob_test = logit_nonlin.predict_proba(X_rbf_feat_test)[:, 1]
prob_sorted = prob_test[feat_test_fill0.index.argsort()]
result_prev["LABEL_Sepsis"] = prob_sorted

In [423]:
result_prev.to_csv("./recall_test_rbf.zip", compression="zip", index=False, float_format='%.3f')

In [404]:
utils.get_score("./train_labels.csv", "./recall_train_rbf.zip")

[0.880140104158019, 0.7682702066333779, 0.7163509452862231, 0.7200754850470517, 0.7163416600616637, 0.7564771589791436, 0.7676717431220972, 0.8042015344518387, 0.7284277753577496, 0.9077601581932911]
[0.6519184425430589, 0.7291738267644755, 0.5417486006038639, 0.7896502307334095]
0.7765716771290455 0.8580173933788627 0.6781227751612019


0.77090394855637

In [435]:
X_rbf_feat_test

array([[ 0.01158714, -0.01357454,  0.00763057, ..., -0.00773793,
         0.00022349, -0.00511733],
       [ 0.01413437,  0.01396206,  0.00843097, ...,  0.01198258,
         0.01410292, -0.00505409],
       [-0.01108782, -0.00990463,  0.01413618, ...,  0.01410968,
         0.01140916,  0.01239361],
       ...,
       [ 0.00422473,  0.0122155 ,  0.01070024, ..., -0.00549259,
         0.00658475,  0.01164897],
       [-0.00915065, -0.00669379,  0.01282676, ...,  0.00559994,
         0.0096538 ,  0.01343743],
       [ 0.00738116, -0.00975196,  0.01136088, ..., -0.00343837,
         0.01059531,  0.01412997]])